In [1]:
import sys
print(sys.executable)
!pip show torch | findstr "Location Version"


d:\vs code\Project4\myvenv\Scripts\python.exe
Version: 2.5.1+cu121
Location: D:\vs code\Project4\myvenv\Lib\site-packages


In [2]:
# check gpu availability and pytorch cuda version
import torch
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA version (built with):", torch.version.cuda)
print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU found")

PyTorch version: 2.5.1+cu121
CUDA available: True
CUDA version (built with): 12.1
GPU name: NVIDIA GeForce RTX 4060 Laptop GPU


In [3]:
# Libraries import 
import torch 
import torch.nn as nn
import torch.optim as optim 
from torch.utils.data import DataLoader,Dataset
from torchvision import transforms, models
from torch.cuda.amp import GradScaler, autocast

import numpy as np
import pandas as pd
from pathlib import Path 
from PIL import Image
from tqdm import tqdm
import json 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report , f1_score
import warnings
warnings.filterwarnings("ignore") 


In [4]:
# Configurations 
class Config:

    # Paths 
    data_dir = Path("../Preprocessing/Data/splits")
    model_save_dir = "models/checkpoints"
    final_model_dir = "models/final"
    logs_dir = "logs"
    results_dir = "results"

    # Model settings 
    model_name = "mobilenet_v2"
    num_classes = 7
    emotions = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

    # Training settings
    batch_size = 64
    learning_rate = 1e-4
    weight_decay = 1e-4
    num_epochs_stage1 = 15 # train head only
    num_epochs_stage2 = 25 # fine-tune entire model

    # image settings 
    img_size = 224

    # training settings 
    num_workers = 4 # for datalaoder (cpu cores)
    mixed_precision = True 

    # Callbacks 
    early_stopping_patience = 7
    lr_scheduler_patience = 4
    lr_scheduler_factor = 0.5

    # Device 
    Device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Random seed 
    seed = 42

def set_seed(seed=42):
# Set random seed for reproducibility
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [5]:
# Step 1: Custom Dataset class 
class EmotionDataset(Dataset):

    def __init__(self, data_dir, split="train", transform=None):

        self.data_dir = Path(data_dir) / split 
        self.transform = transform 
        self.emotions = Config.emotions

        # Build the list of (image path, label index ) tuples 
        self.samples = [] 
        self.labels = []

        for label_idx, emotion in enumerate(self.emotions):
            emotion_dir = self.data_dir / emotion

            if not emotion_dir.exists():
                continue

            # Get all the images in the emotion directory
            image_files = list(emotion_dir.glob("*.jpg")) + list(emotion_dir.glob("*.png"))

            for img_path in image_files:
                self.samples.append(img_path)
                self.labels.append(label_idx)
        print(f"[{split.upper()}] Found {len(self.samples)} samples.")

    def __len__(self):
        " Return the total number of samples "
        return len(self.samples)
    
    def __getitem__(self, idx):
        img_path = self.samples[idx]
        image = Image.open(img_path).convert("RGB")

        # apply transformations
        if self.transform:
            image = self.transform(image)
        
        # get labels 
        label = self.labels[idx]

        return image, label 
    
    def get_class_distribution(self):
        " Returns the distribution of classes in the dataset "
        return np.bincount(self.labels, minlength=Config.num_classes)

In [6]:
# Step 2: Data Augmentation and Transforms
def get_transforms(split='train'):
    """
    Define augmentation pipelines
    
    Train: Aggressive augmentation to prevent overfitting
    Val/Test: Only normalization (no randomness!)
    """
    
    # ImageNet normalization (pretrained models expect this)
    normalize = transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
    
    if split == 'train':
        return transforms.Compose([
            transforms.Resize((Config.img_size, Config.img_size)),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomRotation(degrees=15),
            transforms.ColorJitter(brightness=0.2, contrast=0.2),
            transforms.RandomGrayscale(p=0.1),
            transforms.ToTensor(),
            normalize
        ])
    else:
        return transforms.Compose([
            transforms.Resize((Config.img_size, Config.img_size)),
            transforms.ToTensor(),
            normalize
        ])
 

In [7]:
# Step 3: Compute Class Weights (Handle Imbalnce)

def compute_class_weights(dataset):
    """
    Calculate class weights for imbalanced dataset
    
    Formula: weight = total_samples / (num_classes * samples_per_class)
    Higher weight for minority classes
    """
    class_counts = dataset.get_class_distribution()
    total_samples = len(dataset)
    
    weights = total_samples / (Config.num_classes * class_counts)
    weights = torch.FloatTensor(weights).to(Config.Device)
    
    print("\n📊 Class Distribution & Weights:")
    for i, emotion in enumerate(Config.emotions):
        print(f"   {emotion:10s}: {class_counts[i]:4d} samples (weight: {weights[i]:.3f})")
    
    return weights



In [8]:
class EmotionClassifier(nn.Module):

    def __init__(self, model_name=Config.model_name, num_classes=Config.num_classes, pretrained=True):
        super(EmotionClassifier, self).__init__()

        # Load pretrained backbone
        if model_name == 'mobilenet_v2':
            self.backbone = models.mobilenet_v2(pretrained=pretrained)
            in_features = self.backbone.classifier[1].in_features
            # Remove original classifier
            self.backbone.classifier = nn.Identity()
            
        elif model_name == 'efficientnet_b0':
            self.backbone = models.efficientnet_b0(pretrained=pretrained)
            in_features = self.backbone.classifier[1].in_features
            self.backbone.classifier = nn.Identity()
        
        else:
            raise ValueError(f"Model {model_name} not supported")
        

        # Custom classifier head
        self.classifier = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(in_features, 256),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )
    
    def forward(self, x):
        ''' 
        forward pass 
        Args:
            x: input tensor [batch_size, 3, 224, 224]
            returns logits tensor [batch_size, num_classes]
            '''
        features = self.backbone(x)
        logits = self.classifier(features)
        return logits 
    
    def freeze_backbone(self):
        ''' Freeze backbone layers (for stage 1 training)'''
        for param in self.backbone.parameters():
            param.requires_grad = False
        print("🔒 Backbone frozen. - training head only")
    
    def unfreeze_backbone(self, num_layers=20):
        """Unfreeze last N layers of backbone (for Stage 2 fine-tuning)"""
        # Get all parameters
        params = list(self.backbone.parameters())
        
        # Unfreeze last num_layers
        for param in params[-num_layers:]:
            param.requires_grad = True
        
        frozen = sum(1 for p in self.backbone.parameters() if not p.requires_grad)
        trainable = sum(1 for p in self.backbone.parameters() if p.requires_grad)
        print(f"✅ Unfrozen last {num_layers} layers")
        print(f"   Frozen: {frozen}, Trainable: {trainable}")

In [9]:
# Fixed Training Loop
class Trainer:
    '''
    Complete Training pipeline with pytorch
    '''
    def __init__(self, model, train_loader, val_loader, class_weights, optimizer=None, scheduler=None):
        self.model = model.to(Config.Device)
        self.train_loader = train_loader 
        self.val_loader = val_loader 

        # Loss function with class weights 
        self.criterion = nn.CrossEntropyLoss(weight=class_weights)

        # Optimizer and scheduler (can be set later)
        self.optimizer = optimizer
        self.scheduler = scheduler

        # Mixed precision scaler 
        self.scaler = GradScaler() if Config.mixed_precision else None 

        # Tracking 
        self.history = {
            'train_loss': [], 'train_acc': [],
            'val_loss': [], 'val_acc': []
        }

        self.best_val_acc = 0.0 
        self.best_val_loss = float('inf')
        self.patience_counter = 0  # FIXED: Was early_stopping_counter
    
    def set_optimizer(self, optimizer, scheduler=None):
        """Set or update optimizer and scheduler"""
        self.optimizer = optimizer
        self.scheduler = scheduler

    def train_epoch(self):
        ''' Train for one epoch '''
        self.model.train()

        running_loss = 0.0
        correct_preds = 0
        total_samples = 0

        pbar = tqdm(self.train_loader, desc="Training")

        for images, labels in pbar:
            # Move to gpu 
            images = images.to(Config.Device)
            labels = labels.to(Config.Device)

            # Zero gradients 
            self.optimizer.zero_grad()

            # Forward pass (with mixed precision if enabled)
            if Config.mixed_precision:
                with autocast():
                    outputs = self.model(images)
                    loss = self.criterion(outputs, labels)
                
                # Backward pass
                self.scaler.scale(loss).backward()
                self.scaler.step(self.optimizer)
                self.scaler.update()
            else:
                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()
            
            # Statistics - FIXED: Changed variable names
            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            total_samples += labels.size(0)  # FIXED: was 'total'
            correct_preds += (predicted == labels).sum().item()  # FIXED: was 'correct'
            
            # Update progress bar
            pbar.set_postfix({
                'loss': loss.item(),
                'acc': 100. * correct_preds / total_samples  # FIXED: variable names
            })
        
        epoch_loss = running_loss / total_samples  # FIXED: was 'total'
        epoch_acc = 100. * correct_preds / total_samples  # FIXED: was 'correct/total'
        
        return epoch_loss, epoch_acc
    
    def validate(self):
        self.model.eval()

        running_loss = 0.0 
        correct_preds = 0
        total_samples = 0

        with torch.no_grad():
            for images, labels in tqdm(self.val_loader, desc='Validation'):
                images = images.to(Config.Device)
                labels = labels.to(Config.Device)

                outputs = self.model(images)
                loss = self.criterion(outputs, labels)

                running_loss += loss.item() * images.size(0)
                _, predicted = torch.max(outputs, 1)
                total_samples += labels.size(0)
                correct_preds += (predicted == labels).sum().item()
                
        epoch_loss = running_loss / total_samples
        epoch_acc = 100. * correct_preds / total_samples

        return epoch_loss, epoch_acc
    
    def train(self, num_epochs, stage_name='stage1'):
        """
        Complete training loop
        
        Args:
            num_epochs: Number of epochs to train
            stage_name: 'stage1' or 'stage2' for checkpoint naming
        """
        print(f"\n{'='*60}")
        print(f"🚀 Starting {stage_name.upper()} Training")
        print(f"{'='*60}\n")
        
        for epoch in range(num_epochs):
            print(f"\nEpoch {epoch+1}/{num_epochs}")
            print("-" * 40)
            
            # Train
            train_loss, train_acc = self.train_epoch()
            
            # Validate
            val_loss, val_acc = self.validate()
            
            # Scheduler step
            if self.scheduler:
                self.scheduler.step(val_loss)
            
            # Save history
            self.history['train_loss'].append(train_loss)
            self.history['train_acc'].append(train_acc)
            self.history['val_loss'].append(val_loss)
            self.history['val_acc'].append(val_acc)
            
            # Print summary
            print(f"\nTrain Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
            print(f"Val Loss:   {val_loss:.4f} | Val Acc:   {val_acc:.2f}%")
            
            # Save best model
            if val_acc > self.best_val_acc:
                self.best_val_acc = val_acc
                self.best_val_loss = val_loss
                self.save_checkpoint(f'best_{stage_name}.pth')
                print(f"✅ Best model saved! (Val Acc: {val_acc:.2f}%)")
                self.patience_counter = 0
            else:
                self.patience_counter += 1
            
            # Early stopping
            if self.patience_counter >= Config.early_stopping_patience:
                print(f"\n⚠️  Early stopping triggered (patience: {Config.early_stopping_patience})")
                break
        
        print(f"\n{'='*60}")
        print(f"✅ {stage_name.upper()} Training Complete!")
        print(f"   Best Val Acc: {self.best_val_acc:.2f}%")
        print(f"{'='*60}\n")
    
    def save_checkpoint(self, filename):
        """Save model checkpoint"""
        Path(Config.model_save_dir).mkdir(parents=True, exist_ok=True)
        
        checkpoint = {
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'best_val_acc': self.best_val_acc,
            'best_val_loss': self.best_val_loss,
            'history': self.history,
            'config': {
                'model_name': Config.model_name,
                'num_classes': Config.num_classes,
                'emotions': Config.emotions
            }
        }
        
        path = Path(Config.model_save_dir) / filename
        torch.save(checkpoint, path)
    
    def load_checkpoint(self, filename):
        """Load model checkpoint"""
        path = Path(Config.model_save_dir) / filename
        checkpoint = torch.load(path)
        
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        self.best_val_acc = checkpoint['best_val_acc']
        self.history = checkpoint['history']
        
        print(f"✅ Checkpoint loaded: {filename}")
        print(f"   Best Val Acc: {self.best_val_acc:.2f}%")


In [10]:
def evaluate_model(model, test_loader, save_dir='results'):
    """
    Comprehensive model evaluation
    
    Returns:
        - Accuracy, Precision, Recall, F1
        - Confusion matrix
        - Per-class metrics
    """
    model.eval()
    
    all_preds = []
    all_labels = []
    
    print("\n🧪 Evaluating model on test set...")
    
    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc='Testing'):
            images = images.to(Config.Device)
            
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.numpy())
    
    # Convert to numpy
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    
    # Overall accuracy
    accuracy = 100. * (all_preds == all_labels).sum() / len(all_labels)
    
    # Classification report
    report = classification_report(
        all_labels, all_preds,
        target_names=Config.emotions,
        output_dict=True
    )
    
    # Confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    
    # Print results
    print("\n" + "="*60)
    print("📊 EVALUATION RESULTS")
    print("="*60)
    print(f"\nOverall Accuracy: {accuracy:.2f}%")
    print("\nPer-Class Metrics:")
    print("-" * 60)
    
    for emotion in Config.emotions:
        metrics = report[emotion]
        print(f"{emotion:10s} - Precision: {metrics['precision']:.3f} | "
              f"Recall: {metrics['recall']:.3f} | F1: {metrics['f1-score']:.3f}")
    
    # Save confusion matrix plot
    Path(save_dir).mkdir(parents=True, exist_ok=True)
    plot_confusion_matrix(cm, save_path=f"{save_dir}/confusion_matrix.png")
    
    # Save metrics to JSON
    with open(f"{save_dir}/metrics.json", 'w') as f:
        json.dump({
            'accuracy': accuracy,
            'classification_report': report
        }, f, indent=4)
    
    print(f"\n✅ Results saved to {save_dir}/")
    
    return accuracy, report, cm


def plot_confusion_matrix(cm, save_path):
    """Plot and save confusion matrix"""
    plt.figure(figsize=(10, 8))
    
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=Config.emotions,
                yticklabels=Config.emotions)
    
    plt.title('Confusion Matrix', fontsize=16)
    plt.ylabel('True Label', fontsize=12)
    plt.xlabel('Predicted Label', fontsize=12)
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"   Confusion matrix saved: {save_path}")


def plot_training_history(history, save_path='results/training_history.png'):
    """Plot training curves"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Loss
    ax1.plot(history['train_loss'], label='Train Loss', marker='o')
    ax1.plot(history['val_loss'], label='Val Loss', marker='s')
    ax1.set_title('Loss Over Epochs', fontsize=14)
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Accuracy
    ax2.plot(history['train_acc'], label='Train Acc', marker='o')
    ax2.plot(history['val_acc'], label='Val Acc', marker='s')
    ax2.set_title('Accuracy Over Epochs', fontsize=14)
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy (%)')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"   Training history saved: {save_path}")


In [11]:
def main():
    """Main training pipeline - Two-stage transfer learning"""
    
    # Setup
    set_seed(Config.seed)
    Path(Config.model_save_dir).mkdir(parents=True, exist_ok=True)
    Path(Config.results_dir).mkdir(parents=True, exist_ok=True)
    
    print("="*60)
    print("🚀 EMOTION DETECTION - PyTorch Training Pipeline")
    print("="*60)
    print(f"\n📍 Device: {Config.Device}")
    print(f"📍 Model: {Config.model_name}")
    print(f"📍 Batch Size: {Config.batch_size}")
    print(f"📍 Mixed Precision: {Config.mixed_precision}")
    
    # Check GPU
    if torch.cuda.is_available():
        print(f"\n🎮 GPU Info:")
        print(f"   Name: {torch.cuda.get_device_name(0)}")
        print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    
    # ========================================================================
    # LOAD DATA
    # ========================================================================
    
    print("\n" + "="*60)
    print("📂 Loading Datasets")
    print("="*60 + "\n")
    
    train_dataset = EmotionDataset(
        Config.data_dir, 
        split='train',
        transform=get_transforms('train')
    )
    
    val_dataset = EmotionDataset(
        Config.data_dir,
        split='val',
        transform=get_transforms('val')
    )
    
    test_dataset = EmotionDataset(
        Config.data_dir,
        split='test',
        transform=get_transforms('test')
    )
    
    # Create DataLoaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=Config.batch_size,
        shuffle=True,
        num_workers=Config.num_workers,
        pin_memory=True  # Faster GPU transfer
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=Config.batch_size,
        shuffle=False,
        num_workers=Config.num_workers,
        pin_memory=True
    )
    
    test_loader = DataLoader(
        test_dataset,
        batch_size=Config.batch_size,
        shuffle=False,
        num_workers=Config.num_workers,
        pin_memory=True
    )
    
    # Compute class weights
    class_weights = compute_class_weights(train_dataset)
    
    # ========================================================================
    # BUILD MODEL
    # ========================================================================
    
    print("\n" + "="*60)
    print("🏗️  Building Model")
    print("="*60 + "\n")
    
    model = EmotionClassifier(
        model_name=Config.model_name,
        num_classes=Config.num_classes,
        pretrained=True
    )
    
    # Count parameters
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,}")
    
# ========================================================================
# STAGE 1: TRAIN HEAD ONLY
# ========================================================================

    model.freeze_backbone()

    # Create trainer WITHOUT optimizer initially
    trainer = Trainer(model, train_loader, val_loader, class_weights)

    # Setup optimizer for head training
    optimizer_stage1 = optim.Adam(
        model.classifier.parameters(),
        lr=Config.learning_rate,
        weight_decay=Config.weight_decay
    )

    # FIXED: Removed verbose=True (not supported in PyTorch 2.x+)
    scheduler_stage1 = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer_stage1,
        mode='min',
        factor=Config.lr_scheduler_factor,
        patience=Config.lr_scheduler_patience
    )

    # Set optimizer and scheduler
    trainer.set_optimizer(optimizer_stage1, scheduler_stage1)

    # Train stage 1
    trainer.train(Config.num_epochs_stage1, stage_name='stage1')

    # ========================================================================
    # STAGE 2: FINE-TUNING
    # ========================================================================

    print("\n" + "="*60)
    print("🔓 Unfreezing backbone for fine-tuning")
    print("="*60 + "\n")

    model.unfreeze_backbone(num_layers=30)

    # Setup optimizer for fine-tuning (lower learning rate!)
    optimizer_stage2 = optim.Adam(
        model.parameters(),
        lr=Config.learning_rate / 10,  # 10x smaller LR
        weight_decay=Config.weight_decay
    )

    # FIXED: Removed verbose=True
    scheduler_stage2 = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer_stage2,
        mode='min',
        factor=Config.lr_scheduler_factor,
        patience=Config.lr_scheduler_patience
    )

    # Update optimizer and scheduler for stage 2
    trainer.set_optimizer(optimizer_stage2, scheduler_stage2)

    # Reset patience counter
    trainer.patience_counter = 0

    # Train stage 2
    trainer.train(Config.num_epochs_stage2, stage_name='stage2')

    # ========================================================================
    # EVALUATION
    # ========================================================================
    
    # Load best model
    best_checkpoint = Path(Config.model_save_dir) / 'best_stage2.pth'
    checkpoint = torch.load(best_checkpoint)
    model.load_state_dict(checkpoint['model_state_dict'])
    
    # Evaluate
    accuracy, report, cm = evaluate_model(model, test_loader, Config.results_dir)
    
    # Plot training history
    plot_training_history(trainer.history, f"{Config.results_dir}/training_history.png")
    
    # ========================================================================
    # SAVE FINAL MODEL
    # ========================================================================
    
    print("\n" + "="*60)
    print("💾 Saving Final Model")
    print("="*60 + "\n")
    
    Path(Config.final_model_dir).mkdir(parents=True, exist_ok=True)
    
    # Save full model
    final_path = Path(Config.final_model_dir) / 'emotion_model.pth'
    torch.save({
        'model_state_dict': model.state_dict(),
        'config': {
            'model_name': Config.model_name,
            'num_classes': Config.num_classes,
            'emotions': Config.emotions
        },
        'test_accuracy': accuracy
    }, final_path)
    
    print(f"✅ Model saved: {final_path}")
    
    # Save label mapping
    label_map = {i: emotion for i, emotion in enumerate(Config.emotions)}
    with open(Path(Config.final_model_dir) / 'label_map.json', 'w') as f:
        json.dump(label_map, f, indent=4)
    
    print(f"✅ Label map saved: {Config.final_model_dir}/label_map.json")
    
    print("\n" + "="*60)
    print("🎉 TRAINING PIPELINE COMPLETE!")
    print("="*60)
    print(f"\n📊 Final Test Accuracy: {accuracy:.2f}%")
    print(f"📁 Results saved in: {Config.results_dir}/")
    print(f"💾 Model saved in: {Config.final_model_dir}/")

In [ ]:
if __name__ == "__main__":
    main()


🚀 EMOTION DETECTION - PyTorch Training Pipeline

📍 Device: cuda
📍 Model: mobilenet_v2
📍 Batch Size: 64
📍 Mixed Precision: True

🎮 GPU Info:
   Name: NVIDIA GeForce RTX 4060 Laptop GPU
   Memory: 8.59 GB

📂 Loading Datasets

[TRAIN] Found 27002 samples.
[VAL] Found 5786 samples.
[TEST] Found 5787 samples.

📊 Class Distribution & Weights:
   angry     : 3529 samples (weight: 1.093)
   disgust   :  311 samples (weight: 12.403)
   fear      : 2868 samples (weight: 1.345)
   happy     : 6979 samples (weight: 0.553)
   neutral   : 5365 samples (weight: 0.719)
   sad       : 5042 samples (weight: 0.765)
   surprise  : 2908 samples (weight: 1.326)

🏗️  Building Model

Total parameters: 2,585,607
Trainable parameters: 2,585,607
🔒 Backbone frozen. - training head only

🚀 Starting STAGE1 Training


Epoch 1/15
----------------------------------------


Training:   0%|          | 0/422 [00:00<?, ?it/s]